In [0]:
network = 'VGG'

In [0]:

from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

**IMPORTS AND FUNCTIONS**

In [0]:
from keras.applications.vgg16 import  VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Flatten
from keras.optimizers import SGD
from keras.callbacks import Callback, CSVLogger, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import numpy as np

from tqdm import tqdm


import pandas as pd

Using TensorFlow backend.


In [0]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from scipy.spatial.distance import cdist
from sklearn.mixture import GaussianMixture

**DOWNLOAD FILES**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


VGG

**CNN HELP FUNCTIONS**

In [0]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


def preprocess_input_vgg(x):
      """Wrapper around keras.applications.vgg16.preprocess_input()
      to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
      `preprocessing_function` argument.

      Parameters
      ----------
      x : a numpy 3darray (a single image to be preprocessed)

      Note we cannot pass keras.applications.vgg16.preprocess_input()
      directly to to keras.preprocessing.image.ImageDataGenerator's
      `preprocessing_function` argument because the former expects a
      4D tensor whereas the latter expects a 3D tensor. Hence the
      existence of this wrapper.

      Returns a numpy 3darray (the preprocessed image).

      """
      from keras.applications.vgg16 import preprocess_input
      X = np.expand_dims(x, axis=0)
      X = preprocess_input(X)
      return X[0]


def get_VGG_data_generator(file_num, file_name):
  print('VGG DATA GENERATOR')


  import os
  import zipfile
#   local_zip = 'gdrive/My Drive/Colab Notebooks/Sint_Maarten/' + 'shape/' + str(10) + '/'+ file_name
  local_zip = 'gdrive/My Drive/Colab Notebooks/Sint_Maarten/' + 'material/' + str(10) + '/'+ file_name

#   local_zip = 'gdrive/My Drive/Colab Notebooks/UC_Merced/' + file_name

  print(local_zip)
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/tmp')
  zip_ref.close()

  base_dir = '/tmp/' + file_name.split(".")[0]
  # base_dir = '/tmp/UC_Merced_filtered_45'

  print(base_dir)

  train_dir = os.path.join(base_dir, 'train')
  validation_dir = os.path.join(base_dir, 'validation')
  test_dir = os.path.join(base_dir, 'test')


  # Adding rescale, rotation_range, width_shift_range, height_shift_range,
  # shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
  train_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input_vgg,
      rotation_range=180,
      horizontal_flip=True,
      vertical_flip=True
  )
  val_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input_vgg,
      rotation_range=180,
      horizontal_flip=True,
      vertical_flip=True
  )

  # Note that the validation and test data should not be augmented!
  test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)

  # Flow training images in batches of 16 using train_datagen generator
  train_generator = train_datagen.flow_from_directory(
          train_dir,  # This is the source directory for training images
          target_size=(224, 224),  # All images will be resized to 150x150
          batch_size=16,
          # Since we use binary_crossentropy loss, we need binary labels
          class_mode='sparse', 
          shuffle=True)

  # Flow validation images in batches of 16 using test_datagen generator
  validation_generator = val_datagen.flow_from_directory(
          validation_dir,
          target_size=(224, 224),
          batch_size=16,
          class_mode='sparse',
           shuffle=True)

  # Flow test images in batches of 16 using test_datagen generator
  test_generator = test_datagen.flow_from_directory(
          test_dir,
          target_size=(224, 224),
          batch_size=100,
          class_mode='sparse')
  
  return train_generator, validation_generator, test_generator


In [0]:
def get_cnn_features(generator, model):
  
  features = []
  labels = []
  for i in range(int(np.ceil(generator.samples/generator.batch_size))):
    images, label_indices = generator.next()
    images = model.predict(images)

    for image, label in zip(images, label_indices):
      features.append(image)
      labels.append(label)

  features = np.asarray(features)
  labels = np.asarray(labels)

  return features, labels

In [0]:
def set_data_generators(file_num, network, file_name):
  
  if network == 'VGG':
    train_generator, validation_generator, test_generator = get_VGG_data_generator(file_num, file_name)
    
  elif network == 'Inception':
    train_generator, validation_generator, test_generator = get_Inception_data_generator(file_num, file_name)
    
  elif network == 'Xception':
    train_generator, validation_generator, test_generator = get_Xception_data_generator(file_num, file_name)
  
    
  return train_generator, validation_generator, test_generator

**EVALUATION FUNCTIONS**

In [0]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, cohen_kappa_score

def eval(prediction_lists, label_lists):

  precision_list = []
  recall_list = []
  fscore_list = []
  support_list = []
  accuracy_list =[]
  kappa_list = []



  for prediction_list, label_list in zip(prediction_lists, label_lists): 

    precision, recall, fscore, support= precision_recall_fscore_support(label_list, prediction_list)
    accuracy = accuracy_score(label_list, prediction_list)
    kappa = cohen_kappa_score(label_list, prediction_list)

    precision_list.append(precision)
    recall_list.append(recall)
    fscore_list.append(fscore)
    support_list.append(support)
    accuracy_list.append(accuracy)
    kappa_list.append(kappa)

  precision_list = np.asarray(precision_list)
  recall_list = np.asarray(recall_list)
  fscore_list = np.asarray(fscore_list)
  support_list = np.asarray(support_list)
  accuracy_list = np.asarray(accuracy_list)
  kappa_list = np.asarray(kappa_list) 

  precision = [np.mean(precision_list, axis=0), np.std(precision_list, axis=0)]
  recall = [np.mean(recall_list, axis=0),  np.std(recall_list, axis=0)]
  fscore = [np.mean(fscore_list, axis=0), np.std(fscore_list, axis=0)]
  support = [np.mean(support_list, axis=0), np.std(support_list, axis=0)]
  accuracy = [np.mean(accuracy_list, axis=0), np.std(accuracy_list, axis=0)]
  kappa = [np.mean(kappa_list, axis=0),  np.std(kappa_list, axis=0)]

  return precision, recall, fscore, support, accuracy, kappa

**CODING FUNCTIONS**

In [0]:

def get_BOW_features(images, kmeans):

  histo = np.zeros((images.shape[0], kmeans.n_clusters))
  for i, image in tqdm(enumerate(images)):
    image = image.reshape(np.prod(image.shape[:-1]), image.shape[-1])
    image = normalize(image)
    words = kmeans.predict(image)
    for word in words:
      histo[i][word] += 1
      
  histo = normalize(histo)  
  
  return histo

In [0]:
def get_VLAD_features(images, kmeans):
  
  centers = kmeans.cluster_centers_
  k=kmeans.n_clusters
  V_list = []
  for image in (images):
    image = image.reshape(np.prod(image.shape[:-1]), image.shape[-1])
    m, d = image.shape
    words = kmeans.predict(image)


    V=np.zeros([k,d])

    for i in range(k):

      if np.sum(words == i) > 0:
        V[i]=np.sum(image[words==i,:]-centers[i],axis=0)


    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization

    V = V/np.sqrt(np.dot(V,V))

    V_list.append(V)

  V_list = np.asarray(V_list) 
  
  return V_list


In [0]:
def get_LLC_features(images, kmeans):
  
  LLC_features = []
  for X in (images):
    X = X.reshape(np.prod(X.shape[:-1]), X.shape[-1])
    X = normalize(X)

    C = kmeans.cluster_centers_
    N, d = X.shape
    c, _ = C.shape
    k = int(0.1*c)
    beta = 1e-6
    

    D = cdist(X, C, 'euclidean')
    I = np.zeros((N, k),dtype=int)

    for i in range(N):
      d = D[i, :]
      idx = np.argsort(d)
      I[i, :] = idx[:k]


      II = np.eye(k)

      G = np.zeros((N, c)) #Gammas
      ones = np.ones((k,1))

    for i in range(N):
      idx = I[i,:]
      z = C[idx,:] - np.tile(X[i,:], (k, 1))  # shift ith pt to origin
      Z = np.dot(z,z.T)                       # local covariance
      Z = Z + II*beta*np.trace(Z);            # regularlization (K>D)
      w = np.linalg.solve(Z,ones)     #np.dot(np.linalg.inv(Z), ones)
      w = w/np.sum(w)                         # enforce sum(w)=1       
      G[i,idx] = w.ravel()
#       Y = np.dot(G,C)
    LLC_features.append(G)
    
  LLC_features = np.asarray(LLC_features)
  LLC_features= LLC_features.reshape(LLC_features.shape[0], np.prod(LLC_features.shape[1:]))
    
  return LLC_features

In [0]:
 def get_IFK_features(images, gmm):
    
    means = gmm.means_
    weights = gmm.weights_
    n_kernels = gmm.n_components
    covars = gmm.covariances_

    cov_matrices = np.empty(shape=(n_kernels, covars.shape[1], covars.shape[1]))
    for i in range(n_kernels):
      cov_matrices[i, :, :] = np.diag(covars[i, :])
    covars = cov_matrices
    
    n_images = images.shape[0]
    n_iter = int(np.ceil(n_images/10))
    
    fisher_features = []
    for i in (range(1, n_iter + 1)):
      
      up = min(n_images, 10*i)
      X = images[(i-1)*10:up]

      X = X.reshape(1, X.shape[0],np.prod(X.shape[1:3]), X.shape[3])


      assert X.ndim == 4

      n_videos, n_frames = X.shape[0], X.shape[1]

      X = X.reshape((-1, X.shape[-2], X.shape[-1])) #(n_images, n_features, n_feature_dim)
      X_matrix = X.reshape(-1, X.shape[-1])
      X_matrix = normalize(X_matrix)
      

      # set equal weights to predict likelihood ratio
      gmm.weights_ = np.ones(n_kernels) / n_kernels
      likelihood_ratio = gmm.predict_proba(X_matrix).reshape(X.shape[0], X.shape[1], n_kernels)

      var = np.diagonal(covars, axis1=1, axis2=2)

      norm_dev_from_modes = ((X[:,:, None, :] - means[None, None, :, :])/ var[None, None, :, :]) # (n_images, n_features, n_kernels, n_featur_dim)

      # mean deviation
      mean_dev = np.multiply(likelihood_ratio[:,:,:, None], norm_dev_from_modes).mean(axis=1) #(n_images, n_kernels, n_feature_dim)
      mean_dev = np.multiply(1 / np.sqrt(weights[None, :,  None]), mean_dev) #(n_images, n_kernels, n_feature_dim)

      # covariance deviation
      cov_dev = np.multiply(likelihood_ratio[:,:,:, None], norm_dev_from_modes**2 - 1).mean(axis=1)
      cov_dev = np.multiply(1 / np.sqrt(2 *weights[None, :,  None]), cov_dev)

      fisher_vectors = np.concatenate([mean_dev, cov_dev], axis=1)

      # final reshape - separate frames and videos
      assert fisher_vectors.ndim == 3
      fisher_vectors = fisher_vectors.reshape((n_videos, n_frames, fisher_vectors.shape[1], fisher_vectors.shape[2]))


      fisher_vectors = np.sqrt(np.abs(fisher_vectors)) * np.sign(fisher_vectors) # power normalization
      fisher_vectors = fisher_vectors / np.linalg.norm(fisher_vectors, axis=(2,3))[:,:,None,None]

      fisher_vectors[fisher_vectors < 10**-4] = 0
      
      for image in fisher_vectors[0]:
        fisher_features.append(image)
        
        
    fisher_features = np.asarray(fisher_features)
    
    fisher_features = fisher_features.reshape(fisher_features.shape[0], np.prod(fisher_features.shape[1:]))
    
    return fisher_features

**LOAD MODELS**

In [0]:
import os
# PATH = '/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/trained_models/UC_Merced/'
# PATH = '/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/trained_models/Shape/'
PATH = '/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/trained_models/Material/'

model_names = os.listdir(PATH)
model_names.sort()

models = []

for name in model_names:
  
  file_path = os.path.join(PATH, name)
  print(file_path)
  models.append(load_model(file_path))

/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/trained_models/Material/3_roof_material_100_0_9
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/trained_models/Material/3_roof_material_10_0_9
/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/trained_models/Material/3_roof_material_20_0_9
/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/trained_models/Material/3_roof_material_30_0_9
/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/trained_models/Material/3_roof_material_40_0_9
/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/trained_models/Material/3_roof_material_50_0_9
/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/trained_models/Material/3_roof_material_60_0_9
/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/model

**LOAD/CREATE DATAFRMES**

In [0]:
df_file_name = 'coding_df_Material'
PATH_df = '/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/classification/'
if os.path.isfile(PATH_df + df_file_name):
  df = pd.read_pickle(PATH_df + df_file_name)
  print('LOAD DF')
else:
  df = pd.DataFrame(columns=['network', 'coding', 'par1', 'precision', 'recall', 'fscore', 'support', 'accuracy', 'kappa', 'labels', 'predictions'])
  print('CREATE NEW DF')
  
columns = list(df.columns)

LOAD DF


In [0]:
def get_coded_features(coding, images, cluster):
  
  if coding == 'BOW':
    features =  get_BOW_features(images, cluster)
    
  if coding == 'IFK':
    features =  get_IFK_features(images, cluster)
    
  if coding == 'VLAD':
    features =  get_VLAD_features(images, cluster)
    
  if coding == 'LLC':
    features =  get_LLC_features(images, cluster)
    
  return features


def get_parameters(coding):
  if coding == 'BOW':
    parameters =  [1000]
    
  if coding == 'IFK':
    parameters =  [100]
    
  if coding == 'VLAD':
    parameters =  [100]
    
  if coding == 'LLC':
    parameters =  [100]
    
  if coding == 'None':
    parameters =  ['None']
    
    
  return parameters 

In [0]:
import pickle

In [0]:
from sklearn.svm import LinearSVC

for coding in ['IFK']:
  print('CODING: ', coding)
  
  for par1 in get_parameters(coding):
    print('PARAMETER: ', par1 )
    #Predict
    prediction_lists = []
    label_lists = []

    for num in range(len(models)):
#     for num in range(2, 3):
      print('FOLD', num + 1, 'out of', len(models))
      model = models[num]
      model = Model(inputs=model.input, outputs=model.get_layer('block5_conv3').output)

      
      if num == 0 and len(models) ==  10:
        num = 10
        
      #Filename
#       file_name = 'UC_Merced_filtered_'+ str(num) +'.zip'
#       file_name = 'roof_shape_'+ str(num*10) +'_0_9' + '.zip'
      file_name = 'roof_material_'+ str(num*10) +'_0_9' + '.zip'

      #generators
      train_generator, validation_generator, test_generator = set_data_generators(num, network, file_name)
    
      if coding != 'None':

        #get training data
        x_train, y_train = get_cnn_features(train_generator, model)

        print(x_train.shape)
        #local features 
        local_features = x_train
        local_features = np.reshape(local_features, (np.prod(local_features.shape[:-1]), local_features.shape[3]))
        local_features = normalize(local_features)  
      
        print('SET UP ENCODING')
        #clustering
        if coding == 'IFK':
          cluster =  GaussianMixture(n_components=par1, covariance_type='diag', max_iter=1000).fit(local_features)
        else:
          cluster = MiniBatchKMeans(n_clusters=par1,
                                    random_state=0,
                                    batch_size=2000)
        cluster.fit(local_features)

        del local_features

      print('GENERATE FEATURES')
      import time
      time.sleep(1) 
      
      #bovw
      x_train = []
      y_train = []

      for i in range(2):
        x_train_, y_train_ = get_cnn_features(train_generator, model)
        x_val_, y_val_ = get_cnn_features(validation_generator, model)

        x_train_ = np.concatenate((x_train_, x_val_), axis=0)
        y_train_ = np.concatenate((y_train_, y_val_), axis=0)

        for x, y in zip(x_train_, y_train_):
          x_train.append(x)
          y_train.append(y)


      x_train = np.asarray(x_train)
      y_train = np.asarray(y_train)
      
      print('GET CODED FEATURES')
      if coding != 'None':
        x_train = get_coded_features(coding, x_train, cluster)
        
      else: 
        x_train = np.reshape(x_train, (x_train.shape[0], np.prod(x_train.shape[1:])))
        
      print(x_train.shape)
      
      
      print('FIT SVM')
      #classifier
      clf = LinearSVC(max_iter=500*5000, C = 1, multi_class='ovr')
      clf.fit(x_train, y_train)
      
      del x_train
      del y_train
      print('PREDICTIONS')
      #predict
      
      x_test, y_test = get_cnn_features(test_generator, model)

      if coding == 'BOW':


        x_test = get_coded_features(coding, x_test, cluster)

        predictions = clf.predict(x_test)
        labels = y_test      
        
      else:
        
        predictions = []
        labels = []

        for i in tqdm(range(int(np.ceil(test_generator.samples/test_generator.batch_size)))):
          
          
          x_test, y_test = test_generator.next()
          x_test = model.predict(x_test)
          
          if coding == 'None':
            x_test = np.reshape(x_test, (x_test.shape[0], np.prod(x_test.shape[1:])))
          else:
            x_test = get_coded_features(coding, x_test, cluster)


          for pred, label in zip(clf.predict(x_test), y_test):
            predictions.append(pred)
            labels.append(label)

        predictions = np.asarray(predictions)
        labels = np.asarray(labels)
        
        data = [predictions, labels]
        
        
        with open(PATH_df + str(num) + 'filename.pickle', 'wb') as handle:
          pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

      prediction_lists.append(predictions)
      label_lists.append(labels)

    precision, recall, fscore, support, accuracy, kappa =  eval(prediction_lists, label_lists)
    labels = np.asarray(label_lists).flatten()
    predictions = np.asarray(prediction_lists).flatten()

    obs = pd.DataFrame([[network, coding, par1, precision, recall, fscore, support, accuracy, kappa, labels, predictions]], columns=columns)
    df = df.append(obs)
    obs
    
  df.to_pickle(PATH_df + df_file_name)
    
  

CODING:  IFK
PARAMETER:  100
FOLD 3 out of 10
VGG DATA GENERATOR
gdrive/My Drive/Colab Notebooks/Sint_Maarten/material/10/roof_material_20_0_9.zip
/tmp/roof_material_20_0_9
Found 814 images belonging to 3 classes.
Found 351 images belonging to 3 classes.
Found 10496 images belonging to 3 classes.
(814, 14, 14, 512)
SET UP ENCODING
GENERATE FEATURES
GET CODED FEATURES
(2330, 102400)
FIT SVM
PREDICTIONS


100%|██████████| 105/105 [1:02:29<00:00, 35.49s/it]


In [0]:
df.to_pickle(PATH_df + df_file_name)
# df_file_name

In [0]:
df.to_pickle(PATH_df + 'NEW_CODING_MATERIAL')

In [0]:
prediction_lists = []
label_lists = []


for num in range(0,10):
  
  if num == 0:
    num = 10

  with open(PATH_df +str(num) + 'filename.pickle', 'rb') as handle:
      predictions, labels = pickle.load(handle)

  prediction_lists.append(predictions)
  label_lists.append(labels)


precision, recall, fscore, support, accuracy, kappa =  eval(prediction_lists, label_lists)
labels = np.asarray(label_lists, int).flatten()
predictions = np.asarray(prediction_lists, int).flatten()
obs = pd.DataFrame([[network, coding, par1, precision, recall, fscore, support, accuracy, kappa, labels, predictions]], columns=columns)

In [0]:
df = df.append(obs)

In [0]:
# df = df.reset_index().drop([4,5,6]).drop('index', axis=1)

In [0]:
df= df.reset_index().drop(4).drop('index', axis=1).reset_index().drop('index', axis=1)

In [0]:
df

,network,coding,par1,precision,recall,fscore,support,accuracy,kappa,labels,predictions
0,VGG,None,None,"[[0.5999222000346907, 0.7078168332423942, 0.49...","[[0.5312932226832641, 0.7754704630993291, 0.40...","[[0.561654158049029, 0.7395791150298361, 0.441...","[[2892.0, 6111.0, 1493.0], [0.0, 0.0, 0.0]]","[0.6551829268292683, 0.009312339393659967]","[0.366147638339816, 0.021664362504185877]","[1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 2, 0, 2, ...","[1, 2, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 2, 1, 2, ..."
1,VGG,BOW,1000,"[[0.5957306458636051, 0.6977654839490453, 0.53...","[[0.5091632088520055, 0.7962853870070364, 0.37...","[[0.5478961969052729, 0.7435175716736466, 0.43...","[[2892.0, 6111.0, 1493.0], [0.0, 0.0, 0.0]]","[0.6566120426829268, 0.010906050753588917]","[0.35680089814496785, 0.017198931495166235]","[1, 1, 1, 0, 1, 0, 1, 1, 2, 1, 1, 0, 0, 1, 0, ...","[2, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, ..."
2,VGG,VLAD,100,"[[0.6704308674339978, 0.7142208924055418, 0.61...","[[0.533609958506224, 0.8403207331042383, 0.398...","[[0.5913084566801288, 0.7714782418294355, 0.47...","[[2892.0, 6111.0, 1493.0], [0.0, 0.0, 0.0]]","[0.6929401676829269, 0.0074360974947416305]","[0.4167208286594851, 0.012479352727029908]","[0, 2, 0, 2, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 1, ...","[0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 1, ..."
3,VGG,LLC,100,"[[0.5602261577636733, 0.6936128911824869, 0.47...","[[0.5207814661134162, 0.7610374734086074, 0.34...","[[0.539235652889096, 0.7255478258302017, 0.398...","[[2892.0, 6111.0, 1493.0], [0.0, 0.0, 0.0]]","[0.6357755335365853, 0.011473857278443161]","[0.32841680254648453, 0.02173627771118564]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 2, ...","[0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, ..."
4,VGG,IFK,100,"[[0.6680810542442371, 0.6541866027265955, 0.61...","[[0.36002766251728907, 0.9033873343151695, 0.2...","[[0.4660079060457523, 0.7586316419707991, 0.30...","[[2892.0, 6111.0, 1493.0], [0.0, 0.0, 0.0]]","[0.6539348323170732, 0.00870028391845894]","[0.2843886300725028, 0.030575889725215247]","[0, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 0, 2, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 2, 1, 1, ..."
